In [1]:
import pandas as pd
import numpy as np
import os
import glob
import warnings
warnings.filterwarnings(action='ignore')

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup as bs

In [2]:
path='./data/chromedriver.exe'
driver = webdriver.Chrome(service=Service(path))

In [3]:
# 네이버 지도 접속
driver.get('https://map.naver.com/')
time.sleep(3)

In [ ]:
### <for문> 도착지에 세 번째(끝까지) 정류장 번호 입력
# 폴더 내 버스 데이터 모두 불러오기
# df_time_list
file_list = glob.glob('./data/일반좌석구분/미정차/*.xlsx')

df_list = []
for i in range(len(file_list)):
    print(file_list[i])
    # 폴더 내 하나의 파일 불러와서 '정류장번호' string으로 변환 후 리스트 저장
    df_time = pd.read_excel(file_list[i])
    del df_time['Unnamed: 0']
    df_time_list = []
    for data in df_time['정류장번호']:
        df_time_list.append(str(data).zfill(5))
    # 배너 '길찾기 클릭'
    element = driver.find_element(By.CLASS_NAME,"link_navbar.directions")
    element.click()
    time.sleep(2)
    # '자동차' 버튼 클릭
    car_btn = driver.find_element(By.CLASS_NAME,"link_tab.car")
    car_btn.click()
    time.sleep(2)
    # 출발지:첫 번째 정류장 번호 입력 후 ENETER > '경상북도 경주'이면 클릭
    dep_search = driver.find_element(By.ID, 'directionStart0')
    start_searh_name = df_time_list[0]
    if start_searh_name == '00미정차':
        continue
    dep_search.send_keys(start_searh_name)
    dep_search.send_keys(Keys.ENTER)
    time.sleep(3)
    geongbuk_button = driver.find_elements(By.CLASS_NAME, 'search_title')
    for count in range(len(geongbuk_button)):
        if geongbuk_button[count].text == df_time['버스노선'][0]:
            geongbuk_button[count].click()
    time.sleep(3)
    ##### 크롤링 #####
    for j in range(1, len(df_time_list)):
        # 도착지 > 두 번째 정류장 번호 입력 후 ENETER > '경상북도 경주'이면 클릭
        arr_search = driver.find_element(By.ID, 'directionGoal1')
        arr_searh_name = df_time_list[j]
        if arr_searh_name == '00미정차':
            continue
        arr_search.send_keys(arr_searh_name)
        arr_search.send_keys(Keys.ENTER)
        time.sleep(3)
        geongbuk_button = driver.find_elements(By.CLASS_NAME, 'search_title')
        for z in range(len(geongbuk_button)):
            if geongbuk_button[z].text == df_time['버스노선'][0]:
                geongbuk_button[z].click()
        time.sleep(4)
        # 길찾기 버튼 클릭
        route_btn = driver.find_element(By.CLASS_NAME,"btn.btn_direction.active")
        route_btn.click()
        time.sleep(4)
        # '최적'에서 버스 그림에 있는 시간을 저장
        min_time = driver.find_elements(By.CLASS_NAME, "duration_time")
        min_time[0].text
        time.sleep(4)
        # 행 마다 도착시간 컬럼에 크롤링한 도착 시간 저장
        df_time['도착시간'].loc[j] = min_time[0].text
        # 출발지/도착지 전환 버튼 클릭
        change_btn = driver.find_element(By.CLASS_NAME,"btn_switch.ng-star-inserted")
        change_btn.click()
        time.sleep(4)
        # 도착지 이전 입력값 지우기
        arr_search.send_keys(Keys.CONTROL + "A")
        arr_search.send_keys(Keys.DELETE)
        time.sleep(3)
#     파일 이름 리스트로 저장
    name_list = []
    for item in range(len(file_list)):
        name_list.append(file_list[i].split('\\')[1].split('.')[0])
    df_list.append(df_time)
    df_time.to_excel(f"{name_list[i]}"+".xlsx")
    print(df_time)

./data/일반좌석구분/미정차\일반50_3_01036.xlsx
               버스노선  정류장번호  버스번호   도착시간        배차간격
0              신경주역  03142  일반50  05:00  평일 20분 65회
1              옥수마을  03177  일반50     1분  평일 20분 65회
2               광명교    미정차  일반50  05:00  평일 20분 65회
3                광명  01048  일반50     4분  평일 20분 65회
4             남강주유소  01066  일반50     1분  평일 20분 65회
5              와산마을  01260  일반50     1분  평일 20분 65회
6             경주대학교  01016  일반50     1분  평일 20분 65회
7            외외마을입구  01265  일반50     1분  평일 20분 65회
8         베스트원현대서비스  01138  일반50     1분  평일 20분 65회
9         문화중고등학교입구  01126  일반50     1분  평일 20분 65회
10    국민건강보험.정보고등학교  01060  일반50     1분  평일 20분 65회
11  서라벌대학기숙사앞.송화도서관  01187  일반50     1분  평일 20분 65회
12        서라벌대학.신라고  01186  일반50     1분  평일 20분 65회
13          경주초등학교앞  01036  일반50     1분  평일 20분 65회
14          경주여중건너편  01026  일반50     1분  평일 20분 65회
15     고속버스.시외버스터미널  01045  일반50     1분  평일 20분 65회
16      서라벌사거리.황리단길  01190  일반50     1분  평일 20분 65회
17             중앙시장  01309  